In [40]:
import numpy as np
import pandas as pd
from google.colab import drive
import seaborn as sns
import matplotlib.pyplot as plt
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [41]:
DEMO = pd.read_sas('/content/gdrive/My Drive/11º Período/Ciencia de Dados/DEMO_J.XPT', index='SEQN')
DEMO = DEMO[['RIAGENDR', 'RIDAGEYR', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ']]
display(DEMO.head())
DEMO.shape


,RIAGENDR,RIDAGEYR,DMDEDUC2,DMDMARTL,DMDHHSIZ
SEQN,,,,,
93703.0,2.0,2.0,NaN,NaN,5.0
93704.0,1.0,2.0,NaN,NaN,4.0
93705.0,2.0,66.0,2.0,3.0,1.0
93706.0,1.0,18.0,NaN,NaN,5.0
93707.0,1.0,13.0,NaN,NaN,7.0


(9254, 5)

In [42]:
BMX = pd.read_sas('/content/gdrive/My Drive/11º Período/Ciencia de Dados/BMX_J.XPT', index='SEQN')
BMX = BMX[['BMXWT', 'BMXHT', 'BMXBMI']]
display(BMX.head())
BMX.shape

,BMXWT,BMXHT,BMXBMI
SEQN,,,
93703.0,13.7,88.6,17.5
93704.0,13.9,94.2,15.7
93705.0,79.5,158.3,31.7
93706.0,66.3,175.7,21.5
93707.0,45.4,158.4,18.1


(8704, 3)

In [43]:
BPX = pd.read_sas('/content/gdrive/My Drive/11º Período/Ciencia de Dados/BPX_J.XPT', index='SEQN') 
BPX = BPX[['BPXSY1', 'BPXDI1', 'BPXSY2', 'BPXDI2', 'BPXSY3', 'BPXDI3', 'BPXSY4', 'BPXDI4']]
display(BPX.head())
BPX.shape

,BPXSY1,BPXDI1,BPXSY2,BPXDI2,BPXSY3,BPXDI3,BPXSY4,BPXDI4
SEQN,,,,,,,,
93703.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93704.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93705.0,NaN,NaN,NaN,NaN,202.0,62.0,198.0,74.0
93706.0,112.0,74.0,114.0,70.0,108.0,76.0,NaN,NaN
93707.0,128.0,38.0,128.0,46.0,128.0,58.0,NaN,NaN


(8704, 8)

In [44]:
df = pd.merge(DEMO, BMX, left_index=True, right_index=True, how='inner')
df = pd.merge(df, BPX, left_index=True, right_index=True, how='inner')
display(df.head())
df.shape

,RIAGENDR,RIDAGEYR,DMDEDUC2,DMDMARTL,DMDHHSIZ,BMXWT,BMXHT,BMXBMI,BPXSY1,BPXDI1,BPXSY2,BPXDI2,BPXSY3,BPXDI3,BPXSY4,BPXDI4
SEQN,,,,,,,,,,,,,,,,
93703.0,2.0,2.0,NaN,NaN,5.0,13.7,88.6,17.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93704.0,1.0,2.0,NaN,NaN,4.0,13.9,94.2,15.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93705.0,2.0,66.0,2.0,3.0,1.0,79.5,158.3,31.7,NaN,NaN,NaN,NaN,202.0,62.0,198.0,74.0
93706.0,1.0,18.0,NaN,NaN,5.0,66.3,175.7,21.5,112.0,74.0,114.0,70.0,108.0,76.0,NaN,NaN
93707.0,1.0,13.0,NaN,NaN,7.0,45.4,158.4,18.1,128.0,38.0,128.0,46.0,128.0,58.0,NaN,NaN


(8704, 16)

In [45]:
df.RIAGENDR = df.RIAGENDR.replace({1.0: 'Male', 2.0: 'Female'})
df.DMDEDUC2 = df.DMDEDUC2.replace({1.0: '-9th', 2.0: '9-11th', 3.0: 'High School', 4.0: 'Some College', 5.0: 'Graduate+', 7.0: 'Refused', 8.0: 'Don\'t Know'})
df.DMDMARTL = df.DMDMARTL.replace({1.0: 'Married', 2.0: 'Widowed', 3.0: 'Divorced', 4.0: 'Separated', 5.0: 'Never Married', 6.0: 'Living With Partner', 77: 'Refused', 99: 'Don\'t Know'})
df

,RIAGENDR,RIDAGEYR,DMDEDUC2,DMDMARTL,DMDHHSIZ,BMXWT,BMXHT,BMXBMI,BPXSY1,BPXDI1,BPXSY2,BPXDI2,BPXSY3,BPXDI3,BPXSY4,BPXDI4
SEQN,,,,,,,,,,,,,,,,
93703.0,Female,2.0,NaN,NaN,5.0,13.7,88.6,17.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93704.0,Male,2.0,NaN,NaN,4.0,13.9,94.2,15.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93705.0,Female,66.0,9-11th,Divorced,1.0,79.5,158.3,31.7,NaN,NaN,NaN,NaN,202.0,62.0,198.0,74.0
93706.0,Male,18.0,NaN,NaN,5.0,66.3,175.7,21.5,112.0,74.0,114.0,70.0,108.0,76.0,NaN,NaN
93707.0,Male,13.0,NaN,NaN,7.0,45.4,158.4,18.1,128.0,38.0,128.0,46.0,128.0,58.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102952.0,Female,70.0,High School,Married,2.0,49.0,156.5,20.0,136.0,74.0,142.0,78.0,140.0,68.0,NaN,NaN
102953.0,Male,42.0,High School,Separated,1.0,97.4,164.9,35.8,124.0,76.0,122.0,76.0,116.0,74.0,NaN,NaN
102954.0,Female,41.0,Graduate+,Never Married,7.0,69.1,162.6,26.1,116.0,66.0,118.0,72.0,114.0,74.0,NaN,NaN
